In [110]:
import requests
import os
import time
import pickle
from dotenv import load_dotenv
from collections import deque

In [103]:
load_dotenv()

DEFAULT_REGION_CODE = 'NA1'
DEFAULT_REGION = 'americas'
MY_PUUID = 'cgchml5db7FVLW6cfygY7dmICGAp6hU6OPeRX809alh99kdQW0m1I_Uy_RbVbNj0cB6EJZGL0K_5ew'
TEST_MATCH_ID = 'NA1_5031446976'
RATE_LIMITS_PER_SECOND = 20
RIOT_API_KEY = os.getenv("RIOT_API_KEY")

In [58]:
#gets matches from users
def get_user_matches(puuid = None, region=DEFAULT_REGION):
    if not puuid:
        raise Exception('Please Select puuid')
    
    api_url = f'https://{region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids'
    headers = {
        'X-Riot-Token' : RIOT_API_KEY,
    }

    try:
        response = requests.get(api_url,headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f'Error Occured: {e}')

In [77]:
#gets data from match
def get_match_data(match_id = None, region=DEFAULT_REGION):
    if not match_id:
        raise Exception('Please Select Match ID')
    
    api_url = f'https://{region}.api.riotgames.com/lol/match/v5/matches/{match_id}'
    headers = {
        'X-Riot-Token' : RIOT_API_KEY
    }

    try:
        response = requests.get(api_url,headers=headers)
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f'Error Occured: {e}')


In [83]:
#starts with initial puuid, gets recent 20 matches from that puuid, and gets all unique users from those matches, repeat with the new unique users
def get_unique_users(initial_puuid = None, region=DEFAULT_REGION):
    unique_users_set = {initial_puuid}
    unique_users_list = list(unique_users_set)
    current_index = 0
    
    while len(unique_users_set)<=500:
        #cid stands for current user id 
        print(len(unique_users_set))
        cid = unique_users_list[current_index]
        current_index+=1

        try:
            match_history = get_user_matches(cid)
            for match_id in match_history:
                match_data = get_match_data(match_id)
                players = match_data['metadata']['participants']
                for player in players:
                    if player not in unique_users_set:
                        unique_users_set.add(player)
                        unique_users_list.append(player)
        
        except requests.exceptions.RequestException as e:
            print(f'Failed to retrieve data for user {cid}: {e}')
    return list(unique_users_set)

In [84]:
unique_users = get_unique_users(MY_PUUID)
print(len(unique_users))

1
135
300
463
624


In [108]:
def get_unique_matches(unique_users):
    unique_matches_set = set()

    for i,puuid in enumerate(unique_users):
        if i % 50 == 0:
            print(len(unique_matches_set))
        try:
            match_history = get_user_matches(puuid)

            for match_id in match_history:
                unique_matches_set.add(match_id)
                    
        except requests.exceptions.RequestException as e:
            print(f'Failed to retrieve data for user {puuid}: {e}')
        
        time.sleep(1)
    return list(unique_matches_set)

In [109]:
unique_matches = get_unique_matches(unique_users)

0
977
1923
2816
3768
4639
5484
6318
7135
7913
8745
9560
10336


In [111]:
with open('unique_users_1.pkl','wb') as f:
    pickle.dump(unique_users,f)

with open('unique_matches_1.pkl','wb') as f:
    pickle.dump(unique_matches,f)

In [112]:
with open('unique_users.pkl', 'rb') as f:
    unique_users_list = pickle.load(f)

with open('unique_matches.pkl', 'rb') as f:
    unique_matches_list = pickle.load(f)

In [118]:
print(len(unique_users_list))
print(unique_matches_list[0:10])

624
['NA1_5030228074', 'NA1_5029971709', 'NA1_5029303239', 'NA1_5027975587', 'NA1_5031432548', 'NA1_5027868330', 'NA1_5029130552', 'NA1_5031782941', 'NA1_5026770942', 'NA1_5031867356']


In [122]:
match_data = get_match_data('NA1_5029130552')
print(match_data['info']['platformId'])
print(match_data["info"]["gameMode"])
print(match_data["info"]["gameVersion"])
for i in range(len(match_data['metadata']['participants'])):
    print(f'{match_data['info']['participants'][i]['championName']}:{match_data['info']['participants'][i]['win']}')

ARAM
14.13.596.7996
NA1
Vex:False
Olaf:False
Corki:False
Alistar:False
Graves:False
Velkoz:True
Galio:True
Ziggs:True
Gragas:True
Caitlyn:True
